# Import Libraries

In [ ]:
import os
import re
import sys
import time
import math
import faiss
import numpy as np
import pandas as pd
import pickle as pkl
import xgboost as xgb

import tensorflow as tf
from scipy import stats
from keras.callbacks import CSVLogger, TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split

from utils.utils import *
from utils.data_generator import *
from utils.cesnetpreprocessing import generate_balanced_label_and_ID_list_for_labels, load_data_for_ids
from utils.config import get_all_labelled_IDs, get_both_label_indices, get_file_startIds, map_label_num_to_soori_label, picked_fine_classes_25, fine2coarse, CUTOFF_POINT
from models.Model_First import AttentionModelTrunk, build_full_model as build_first_model
from models.Model_Full import build_model as build_full_model
from models.Model_NewRun import build_model as build_newrun_model
from models.Model_NoMB import build_model as build_nomb_model
from models.Model_MoreBlocks import AttentionModelTrunk, build_full_model as build_moreblocks_model

# Prepare Data

In [ ]:
batch_size = 32
dim = CUTOFF_POINT
n_channels = 3
n_classes = len(picked_fine_classes_25)
notebook_output_prefix = './model_outputs/'

In [ ]:
both_label_indices = get_both_label_indices()
all_labelled_IDs = get_all_labelled_IDs()
file_startIds = get_file_startIds()
day_3_endID = int(file_startIds[3][1])
training_x, training_y, perlabel_training_x, perlabel_num = generate_balanced_label_and_ID_list_for_labels(0, day_3_endID, 1000000, 
                                        picked_fine_classes_25, 0, both_label_indices, return_coarse=False)
actual_training_y = [map_label_num_to_soori_label(y) for y in training_y]

In [ ]:
train_indices, val_indices = train_test_split(range(len(training_x)), test_size = 0.20, random_state=19)

training_samples = list(zip(training_x, actual_training_y))
training_exclu_x = [ training_samples[ind][0] for ind in train_indices]
training_exclu_y = [ training_samples[ind][1] for ind in train_indices]

val_x = [ training_samples[ind][0] for ind in val_indices]
val_y = [ training_samples[ind][1] for ind in val_indices]

# Model Training

## The First Model

In [ ]:
# First Model aka Second Run
MODEL_NAME = "FirstModel"
path_prefix = f'{notebook_output_prefix}{MODEL_NAME}'

attention_model = AttentionModelTrunk(name='FirstModelTrunk',
                             num_heads=1, head_size=3, ff_dim=64, num_layers=2, dropout=0.1)
model_first = build_first_model((30,3), attention_model, n_classes, [512], 0.01, 0.01)

training_generator, validation_generator = get_data_generator(training_exclu_x, training_exclu_y, val_x, val_y, n_classes, 
                                   norm_func=normalize("minmax", "cesnet"),feature_select=[0,1,2],
                                   batch_size = 32, dim =(30), n_channels =3)

dummy_init_lr = 1e-8
model_first.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=dummy_init_lr),
    metrics=['accuracy',tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()]
    )

epoch_list = [1, 2, 4, 8, 8]
scheduler2 = scheduler2_factory(epoch_list, [1e-3, 1e-4, 1e-5, 1e-6, 1e-7])

lr_callback = keras.callbacks.LearningRateScheduler(scheduler2)
tensorboard = TensorBoard(log_dir=f'{notebook_output_prefix}logs/', histogram_freq=1, write_graph=True,
                          write_images=True)
checkpointer_loss = ModelCheckpoint(filepath= path_prefix + '_loss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_acc = ModelCheckpoint(monitor='val_accuracy', filepath= path_prefix+ '_acc.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
tensorboard.set_model(model_first)

history = model_first.fit(training_generator, 
          epochs=sum(epoch_list),
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,lr_callback])

In [ ]:
model_first.save_weights(f"{notebook_output_prefix}/model_weights/{MODEL_NAME}.weights.h5")
model_first.save(f"{notebook_output_prefix}/full_model/{MODEL_NAME}.keras")

## The Full Model

In [ ]:
# Full Model
MODEL_NAME = "FullModel"
path_prefix = f'{notebook_output_prefix}{MODEL_NAME}'

model_full = build_full_model(
    (CUTOFF_POINT,3),
    head_size=256,
    num_heads=8,
    ff_dim=64,
    num_transformer_blocks=2,
    mlp_units=[ 512 ],
    mlp_dropout=0.1,
    dropout=0.1,
    transformer_input_size=256,
    n_classes=n_classes
)

training_generator, validation_generator = get_data_generator(training_exclu_x, training_exclu_y, val_x, val_y, n_classes, 
                                   norm_func = normalize("all-three-n2", "cesnet"), feature_select=[0,1,2],
                                   batch_size = 64, dim = (30), n_channels = 3)

dummy_init_lr = 1e-8
model_full.compile(loss='categorical_crossentropy', 
                          optimizer=tf.keras.optimizers.Adam(learning_rate=dummy_init_lr), \
                          metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

epoch_list = [5, 4]
scheduler2 = scheduler2_factory(epoch_list, [1e-7, 1e-4])

lr_callback = keras.callbacks.LearningRateScheduler(scheduler2)
tensorboard = TensorBoard(log_dir=f'{notebook_output_prefix}logs/', histogram_freq=1, write_graph=True,
                          write_images=True)
checkpointer_loss = ModelCheckpoint(monitor='val_loss', filepath= path_prefix + '_loss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_acc = ModelCheckpoint(monitor='val_accuracy', filepath= path_prefix+ '_acc.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_train_loss = ModelCheckpoint(monitor='loss', filepath= path_prefix + '_trainloss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
csv_logger = CSVLogger(path_prefix + '_training.log', append=True)
tensorboard.set_model(model_full)

history1 = model_full.fit(training_generator, 
          epochs=sum(epoch_list),
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,checkpointer_train_loss,lr_callback,csv_logger])

init_lr = 1e-7
lr_callback = keras.callbacks.LearningRateScheduler(scheduler3)
model_full.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=init_lr), 
                   metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

history2 = model_full.fit(training_generator, 
          epochs=31,
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,checkpointer_train_loss,lr_callback,csv_logger])

In [ ]:
model_full.save_weights(f"{notebook_output_prefix}/model_weights/{MODEL_NAME}.weights.h5")
model_full.save(f"{notebook_output_prefix}/full_model/{MODEL_NAME}.keras")

## The New Run Model

In [ ]:
# New Run Model
MODEL_NAME = "NewRunModel"
path_prefix = f'{notebook_output_prefix}{MODEL_NAME}'

model_newrun = build_newrun_model(
    (CUTOFF_POINT,3),
    head_size=3,
    num_heads=8,
    ff_dim=64,
    num_transformer_blocks=2,
    mlp_units=[ 90 ],
    mlp_dropout=0.05,
    dropout=0.1,
    transformer_input_size=3,
    n_classes=n_classes
)

training_generator, validation_generator = get_data_generator(training_exclu_x, training_exclu_y, val_x, val_y, n_classes, 
                                   norm_func=normalize("minmax", "cesnet"),feature_select=[0,1,2],
                                   batch_size = 64, dim =(30), n_channels =3)
    
dummy_init_lr = 1e-8
model_newrun.compile(loss='categorical_crossentropy', 
                          optimizer=tf.keras.optimizers.Adam(learning_rate=dummy_init_lr), \
                          metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

epoch_list = [5, 5, 10, 10, 30]
scheduler2 = scheduler2_factory(epoch_list, [1e-4, 1e-5, 1e-6, 1e-7, 1e-8])

lr_callback = keras.callbacks.LearningRateScheduler(scheduler2)
tensorboard = TensorBoard(log_dir=f'{notebook_output_prefix}logs/', histogram_freq=1, write_graph=True,
                          write_images=True)
checkpointer_loss = ModelCheckpoint(monitor='val_loss', filepath= path_prefix + '_loss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_acc = ModelCheckpoint(monitor='val_accuracy', filepath= path_prefix+ '_acc.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_train_loss = ModelCheckpoint(monitor='loss', filepath= path_prefix + '_trainloss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
csv_logger = CSVLogger(path_prefix + '_training.log', append=True)
tensorboard.set_model(model_newrun)

history = model_newrun.fit(training_generator, 
          epochs=sum(epoch_list),
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,checkpointer_train_loss,lr_callback,csv_logger])

In [ ]:
model_newrun.save_weights(f"{notebook_output_prefix}/model_weights/{MODEL_NAME}.weights.h5")
model_newrun.save(f"{notebook_output_prefix}/full_model/{MODEL_NAME}.keras")

## The NoMB Model

In [ ]:
# NoMB Model
MODEL_NAME = "NoMBModel"
path_prefix = f'{notebook_output_prefix}{MODEL_NAME}'

model_nomb = build_nomb_model(
    (CUTOFF_POINT,3),
    head_size=256,
    num_heads=8,
    ff_dim=64,
    num_transformer_blocks=2,
    mlp_units=[ 512 ],
    mlp_dropout=0.1,
    dropout=0.1,
    transformer_input_size=256,
    n_classes=n_classes
)

training_generator, validation_generator = get_data_generator(training_exclu_x, training_exclu_y, val_x, val_y, n_classes, 
                                   norm_func=normalize("minmax", "cesnet"),feature_select=[0,1,2],
                                   batch_size = 64, dim =(30), n_channels =3)

dummy_init_lr = 1e-8
model_nomb.compile(loss='categorical_crossentropy', 
                          optimizer=tf.keras.optimizers.Adam(learning_rate=dummy_init_lr), \
                          metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

epoch_list = [4, 4, 2]
scheduler2 = scheduler2_factory(epoch_list, [1e-5, 1e-3, 1e-6])

lr_callback = keras.callbacks.LearningRateScheduler(scheduler2)
tensorboard = TensorBoard(log_dir=f'{notebook_output_prefix}logs/', histogram_freq=1, write_graph=True,
                          write_images=True)
checkpointer_loss = ModelCheckpoint(monitor='val_loss', filepath= path_prefix + '_loss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_acc = ModelCheckpoint(monitor='val_accuracy', filepath= path_prefix+ '_acc.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_train_loss = ModelCheckpoint(monitor='loss', filepath= path_prefix + '_trainloss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
csv_logger = CSVLogger(path_prefix + '_training.log', append=True)
tensorboard.set_model(model_nomb)

history1 = model_nomb.fit(training_generator, 
          epochs=sum(epoch_list),
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,checkpointer_train_loss,lr_callback,csv_logger])

init_lr = 1e-5
lr_callback = keras.callbacks.LearningRateScheduler(scheduler3)
model_nomb.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=init_lr), metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
history2 = model_nomb.fit(training_generator, 
          epochs=30,
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,checkpointer_train_loss,lr_callback,csv_logger])

In [ ]:
model_nomb.save_weights(f"{notebook_output_prefix}/model_weights/{MODEL_NAME}.weights.h5")
model_nomb.save(f"{notebook_output_prefix}/full_model/{MODEL_NAME}.keras")

## The More Blocks Model

In [ ]:
# More Blocks Model
MODEL_NAME = "MoreBlocksModel"
path_prefix = f'{notebook_output_prefix}{MODEL_NAME}'

attention_model = AttentionModelTrunk(name='Co3_2x6head_4layer_NoEmbeddingTransformer',
                             num_heads=2, head_size=6, ff_dim=64, num_layers=4, dropout=0.1)
model_moreblocks = build_moreblocks_model((30,3), attention_model, n_classes, [512], 0.01, 0.01)

training_generator, validation_generator = get_data_generator(training_exclu_x, training_exclu_y, val_x, val_y, n_classes, normalize("minmax", "cesnet"),
                                   batch_size = 64,feature_select=[0,1,2], dim =(30), n_channels = 3)

dummy_init_lr = 1e-8
model_moreblocks.compile(loss='categorical_crossentropy', 
                          optimizer=tf.keras.optimizers.Adam(learning_rate=dummy_init_lr), \
                          metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

epoch_list = [3,1,2,10]
scheduler2 = scheduler2_factory(epoch_list, [1e-3, 1e-4, 1e-5,1e-6])

lr_callback = keras.callbacks.LearningRateScheduler(scheduler2)
tensorboard = TensorBoard(log_dir=f'{notebook_output_prefix}logs/', histogram_freq=1, write_graph=True,
                          write_images=True)
checkpointer_loss = ModelCheckpoint(monitor='val_loss', filepath= path_prefix + '_loss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_acc = ModelCheckpoint(monitor='val_accuracy', filepath= path_prefix+ '_acc.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
checkpointer_train_loss = ModelCheckpoint(monitor='loss', filepath= path_prefix + '_trainloss.weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
csv_logger = CSVLogger(path_prefix + '_training.log', append=True)
tensorboard.set_model(model_moreblocks)

history = model_moreblocks.fit(training_generator, 
          epochs=15,
          verbose=1,
          shuffle=False,
          validation_data=validation_generator,
          callbacks=[tensorboard,checkpointer_loss,checkpointer_acc,checkpointer_train_loss,lr_callback,csv_logger])

In [ ]:
model_moreblocks.save_weights(f"{notebook_output_prefix}/model_weights/{MODEL_NAME}.weights.h5")
model_moreblocks.save(f"{notebook_output_prefix}/full_model/{MODEL_NAME}.keras")